Am Beispiel von https://web1.karlsruhe.de/service/Parken/detail.php?id=K01


In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import time

In [2]:
parking_data = pd.DataFrame(columns=["timestamp", "free_spots"])
parking_data.set_index("timestamp", inplace=True)
parking_data

,free_spots
timestamp,


In [4]:
n = 3
sleep_time = 5 * 60  # 5 minutes

for i in range(n):
    try:
        res = requests.get("https://web1.karlsruhe.de/service/Parken/detail.php?id=K01")
        if res.status_code == 200:
            soup = BeautifulSoup(res.text, "html.parser")

            # find h1 where the parking lot name is stored
            parking_lot = soup.find("h1")

            # find element with text "Gesamtzahl der freien Stellplätze"
            free_spots = soup.find(string="Gesamtzahl der freien Stellplätze").find_next(
                "td"
            )

            time_string = parking_lot.find_next("p").find_next("p")  # two after heading
            time_string = time_string.text.split("von")[1].split("Uhr")[0]
            time_string = time_string.split("-")
            time_string = time_string[0].strip() + " " + time_string[1].strip()

            timestamp = pd.to_datetime(time_string, format="%d.%m.%Y %H:%M")

            # append data to dataframe
            parking_data.loc[timestamp] = [
                free_spots.text
            ]
            print("Data added: ", timestamp)
            print(f"   {free_spots.text} free spots")

        else:
            print("Error: ", res.status_code)
    except Exception as e:
        print("Error: ", e)

    time.sleep(sleep_time)


parking_data

Data added:  2023-10-31 17:35:00
   359 free spots
Data added:  2023-10-31 17:40:00
   362 free spots
Data added:  2023-10-31 17:45:00
   363 free spots


,free_spots
timestamp,
2023-10-31 17:30:00,357
2023-10-31 17:35:00,359
2023-10-31 17:40:00,362
2023-10-31 17:45:00,363


In [5]:
parking_data.to_csv("../parking_data.csv")